In [2]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 95.3 MB/s eta 0:00:00


In [3]:
import pulp

## **Ejercicio 1**

# **Función para leer el input:**

In [4]:
def read_input(file):
  #readline lee la primera linea del file
  #split use el espacio para separar en palabras
  #devuelve un arreglo de palbras
    line = file.readline().split()
    n, m = int(line[0]), int(line[1])

  #si lee dos 0 es que ya no hay más instancias
    if n == 0 and m == 0:
        return 0, 0, 0, [], [], {}

  #Lee el costo promedio de entregar un paquete desde el service center
  #Ese valor se llama S
    S = float(file.readline())

  #Defino una lista de capacidad y otra de costo para cada nodo
    capacities = []
    costs = []

  #Itero sobre los n nodos
  #Almaceno su capacidad y costo
    for _ in range(n):
        # Lee capacidad como entero y costo como float
        capacity, cost = file.readline().split()
        capacities.append(int(capacity))
        costs.append(float(cost))

  #Defino un diccionario para la cobertura
    coverage = {}
  #Itero por cada nodo
    for i in range(n):
      #Transformo a cada elemento leido en un entero con map
        data = list(map(int, file.readline().split()))
      #Acá guardo para el nodo i, que paquetes puede entrgar
        coverage[i] = data[1:]

    return n, m, S, capacities, costs, coverage


In [ ]:

# Re-open the file as it was closed by the previous error
file_path = "/content/drive/MyDrive/Optimización/in.txt"
input_file = open(file_path, 'r')

all_instances_data = []

while True:
    n, m, s, node_capacities, node_costs, node_coverage = read_input(input_file)

    if n == 0 and m == 0:
        break

    all_instances_data.append({
        "N": n,
        "M": m,
        "S": s,
        "node_capacities": node_capacities,
        "node_costs": node_costs,
        "node_coverage": node_coverage
    })

input_file.close()

# Optional: Print the prepared data for verification
for i, instance_data in enumerate(all_instances_data):
    print(f"--- Instance {i+1} ---")
    print(f"N: {instance_data['N']}, M: {instance_data['M']}, S: {instance_data['S']}")
    print(f"Node Capacities: {instance_data['node_capacities']}")
    print(f"Node Costs: {instance_data['node_costs']}")
    print(f"Node Coverage: {instance_data['node_coverage']}")
    print("-" * 20)

--- Instance 1 ---
N: 3, M: 8, S: 12.5
Node Capacities: [10, 8, 12]
Node Costs: [5.0, 7.25, 4.5]
Node Coverage: {0: [0, 1, 2, 3, 7], 1: [2, 3, 4, 5], 2: [0, 2, 4, 5, 6, 7]}
--------------------
--- Instance 2 ---
N: 2, M: 5, S: 10.0
Node Capacities: [20, 15]
Node Costs: [4.0, 6.0]
Node Coverage: {0: [0, 1, 2], 1: [0, 1, 3, 4]}
--------------------


# **Procesamiento:**

In [93]:
file_path = "/content/drive/MyDrive/Optimización/in.txt"
input_file = open(file_path, 'r')
n, m, S, capacities, costs, coverage = read_input(input_file)

n, m, S, capacities, costs, coverage = read_input(input_file)

In [66]:
#Considero que el service center es el nodo n+1.
#Como cuentan desde el 0, el nodo n
#Su costo es S
costs.append(S)
#Su capacidad es igual a la cantidad de paquetes a entregar, podría ser superior pero con esto basta
capacities.append(m)
#Su coverage es el de todos los paquetes
coverage[n] = list(range(m))
n = n + 1

In [67]:
model = pulp.LpProblem("Delivery_Cost_Optimization", pulp.LpMinimize)

In [68]:
deliver = pulp.LpVariable.dicts("Deliver_Package_In_Node", [(i, j) for i in range(m) for j in range(n+1)], cat='Binary')

In [69]:
model += pulp.lpSum(pulp.lpSum((j in coverage[i]) * deliver[(j, i)] for j in range(m))*costs[i] for i in range(n)), "Total_Cost"

In [70]:
for i in range(n):
    model += pulp.lpSum(deliver[(j, i)] for j in range(m)) <= capacities[i], f"Capacity_Constraint_{i}"

In [71]:
for j in range(m):
    model += pulp.lpSum(deliver[(j, i)] for i in range(n)) == 1, f"Package_Delivery_Constraint{j}"

In [ ]:
model.solve()

In [ ]:
print(f"Estado de la solución: {pulp.LpStatus[model.status]}")
print("-" * 30)
# El valor del objetivo es el número mínimo de depositos
min_cost = pulp.value(model.objective)
print(f"Costo mínimo: {min_cost}")
print("-" * 30)
print("Paquetes entregados por cada nodo:")

# Imprimimos solo los depositos cuya variable de decisión sea 1
for (j, i) in deliver:
    if pulp.value(deliver[(j, i)]) == 1:
        print(f"  Paquete {j} entregado por Nodo {i}")

In [94]:
# Read input data
file_path = "/content/drive/MyDrive/Optimización/in.txt"
input_file = open(file_path, 'r')
n, m, S, capacities, costs, coverage = read_input(input_file)

# Consider the service center as node n+1
costs.append(S)
capacities.append(m)
coverage[n] = list(range(m))
n = n + 1

model = pulp.LpProblem("Delivery_Cost_Optimization", pulp.LpMinimize)

# Define decision variables
deliver = pulp.LpVariable.dicts(
    "Deliver_Package_In_Node",
    [(j, i) for i in range(n) for j in coverage[i]],  # only valid pairs
    cat='Binary'
)

# Define the objective function
# Create the model
model += pulp.lpSum(
    deliver[(j, i)] * costs[i]
    for i in range(n)
    for j in coverage[i]
), "Total_Cost"


# Define constraints
# Capacity constraint: Each node's capacity is not exceeded
# Capacity: Each node i can only deliver up to its capacity
for i in range(n):
    model += pulp.lpSum(deliver[(j, i)] for j in coverage[i]) <= capacities[i], f"Capacity_{i}"

# Each package j must be delivered by exactly one node
for j in range(m):
    # only sum over nodes i where j ∈ coverage[i]
    model += pulp.lpSum(deliver[(j, i)] for i in range(n) if j in coverage[i]) == 1, f"Package_{j}"


# Solve the problem
model.solve()

print(f"Estado de la solución: {pulp.LpStatus[model.status]}")
print("-" * 30)
# El valor del objetivo es el número mínimo de depositos
min_cost = pulp.value(model.objective)
print(f"Costo mínimo: {min_cost}")
print("-" * 30)
print("Paquetes entregados por cada nodo:")

# Imprimimos solo los depositos cuya variable de decisión sea 1
for (j, i) in deliver:
    if pulp.value(deliver[(j, i)]) == 1:
        print(f"  Paquete {j} entregado por Nodo {i}")

Estado de la solución: Optimal
------------------------------
Costo mínimo: 37.0
------------------------------
Paquetes entregados por cada nodo:
  Paquete 1 entregado por Nodo 0
  Paquete 3 entregado por Nodo 0
  Paquete 0 entregado por Nodo 2
  Paquete 2 entregado por Nodo 2
  Paquete 4 entregado por Nodo 2
  Paquete 5 entregado por Nodo 2
  Paquete 6 entregado por Nodo 2
  Paquete 7 entregado por Nodo 2


# **Todo Junto**

In [119]:
def read_input(file):
  #readline lee la primera linea del file
  #split use el espacio para separar en palabras
  #devuelve un arreglo de palbras
    line = file.readline().split()
    n, m = int(line[0]), int(line[1])

  #si lee dos 0 es que ya no hay más instancias
    if n == 0 and m == 0:
        return 0, 0, 0, [], [], {}

  #Lee el costo promedio de entregar un paquete desde el service center
  #Ese valor se llama S
    S = float(file.readline())

  #Defino una lista de capacidad y otra de costo para cada nodo
    capacities = []
    costs = []

  #Itero sobre los n nodos
  #Almaceno su capacidad y costo
    for _ in range(n):
        # Lee capacidad como entero y costo como float
        capacity, cost = file.readline().split()
        capacities.append(int(capacity))
        costs.append(float(cost))

  #Defino un diccionario para la cobertura
    coverage = {}
  #Itero por cada nodo
    for i in range(n):
      #Transformo a cada elemento leido en un entero con map
        data = list(map(int, file.readline().split()))
      #Acá guardo para el nodo i, que paquetes puede entrgar
        coverage[i] = data[1:]

    return n, m, S, capacities, costs, coverage

#####################################################################

file_path = "/content/drive/MyDrive/Optimización/in.txt"
input_file = open(file_path, 'r')
solutions = []

while True:
  # Read input data
  n, m, S, capacities, costs, coverage = read_input(input_file)

  if n == 0:
    break

  # Considero al SVC el nodo n+1 con costo S, capacidad m (PODRIA SER MAYOR) y coverage de todos los paquetes
  costs.append(S)
  capacities.append(m)
  coverage[n] = list(range(m))
  n = n + 1

  model = pulp.LpProblem("Delivery_Cost_Optimization", pulp.LpMinimize)

  # Defino las variables, cada variable representa entrage el paquete j en el nodo i. Solo si es posible.
  deliver = pulp.LpVariable.dicts("Deliver_Package_In_Node",[(j, i) for i in range(n) for j in coverage[i]], cat='Binary')

  #Quiero minimzar el costo de entregar los paquetes, para eso tengo que elegir donde los entrego
  model += pulp.lpSum(deliver[(j, i)] * costs[i] for i in range(n) for j in coverage[i]), "Total_Cost"

  #Cada nodo no puede entrgar más paquetes que su capacidad
  for i in range(n):
      model += pulp.lpSum(deliver[(j, i)] for j in coverage[i]) <= capacities[i], f"Capacity_{i}"

  #Cada paquete j tiene que ser entrgado a un único nodo
  for j in range(m):
      #Ese nodo tiene que poder entrgarlo
      model += pulp.lpSum(deliver[(j, i)] for i in range(n) if j in coverage[i]) == 1, f"Package_{j}"


  # Resuelvo
  model.solve()
  solutions.append((model, deliver, n, m, coverage))

  ################################################################################
with open("out.txt", "w") as f:
  for instance_index, (model, deliver, n, m, coverage) in enumerate(solutions, start=1):
      f.write(f"Caso {instance_index}\n")
      f.write(f"{pulp.value(model.objective):.2f}\n")
      for j in range(m):
        for i in range(n):
          if j in coverage[i] and pulp.value(deliver[(j, i)]) == 1:
            r = i if i != n-1 else -1
            f.write(f"{j} {r}\n")
            break
